In [ ]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load dos dados do ficheiro JSON e preparação dos dados para treino e teste
def load_dataset(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    captured_data = data['capturedData']

    accelerometerXAxis = [entry['accelerometerXAxis'] for entry in captured_data]
    accelerometerYAxis = [entry['accelerometerYAxis'] for entry in captured_data]
    accelerometerZAxis = [entry['accelerometerZAxis'] for entry in captured_data]
    gyroscopeXAxis = [entry['gyroscopeXAxis'] for entry in captured_data]
    gyroscopeYAxis = [entry['gyroscopeYAxis'] for entry in captured_data]
    gyroscopeZAxis = [entry['gyroscopeZAxis'] for entry in captured_data]

    # criar dataFrame 
    df = pd.DataFrame({
        'accelerometerXAxis': accelerometerXAxis,
        'accelerometerYAxis': accelerometerYAxis,
        'accelerometerZAxis': accelerometerZAxis,
        'gyroscopeXAxis': gyroscopeXAxis,
        'gyroscopeYAxis': gyroscopeYAxis,
        'gyroscopeZAxis': gyroscopeZAxis
    })

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df.values)

    sequence_length = 11
    sequences = []
    for i in range(len(scaled_data) - sequence_length):
        sequences.append(scaled_data[i:i+sequence_length])

    sequences = np.array(sequences)

    # conjuntos de treino e teste
    X = sequences[:, :-1]
    y = sequences[:, -1]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test, scaler

#treinar o modelo LSTM
def build_and_train_model(X_train, X_test, y_train, y_test):
    # construção do LSTM
    model = Sequential()
    model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(X_train.shape[2])) 
    model.compile(optimizer='adam', loss='mse')

    model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

    return model

# prever os valores e mostrar a comparação
def predict_and_compare(model, X_test, y_test, scaler):
    y_pred = model.predict(X_test)

    y_pred_inverse = scaler.inverse_transform(y_pred)
    y_test_inverse = scaler.inverse_transform(y_test)

    comparison_df = pd.DataFrame({'Real': y_test_inverse.flatten(), 'Predicted': y_pred_inverse.flatten()})

    difference = np.abs(comparison_df['Real'] - comparison_df['Predicted'])
    comparison_df['Absolute Difference'] = difference

    return comparison_df

def classify_acceleration(data):
    max_acceleration_0_10= 0.07
    
    df = pd.DataFrame(data['capturedData'])
    
    df['createdAt'] = pd.to_datetime(df['createdAt'])

    # converte para float
    df['speed Km/h'] = df['speed Km/h'].astype(float)
    
    # calcula a aceleracao com base na mudança de velocidade
    df['acceleration'] = df['speed Km/h'].diff() / df['createdAt'].diff().dt.total_seconds()

    # critérios para classificação 
    slow_threshold = 0.06 * max_acceleration_0_10
    aggressive_threshold = 0.08 * max_acceleration_0_10

    # funcao para classificar a aceleração
    def classify(acceleration):
        if acceleration >= aggressive_threshold:
            return 'aggressive'
        elif acceleration >= slow_threshold:
            return 'normal'
        else:
            return 'slow'

    # aplicar a função de classificação
    df['classification'] = df['acceleration'].apply(classify)

    return df[['acceleration', 'classification']]

def main():
    datasets = [
        # BMW e36 datasets 10m
        r'..\Datasets\Acceleration\Till reached distance\10m\aggressive\BMW e36\acceleration 0m 10m aggressive-23-04-2023-19-42-30.json',
        r'..\Datasets\Acceleration\Till reached distance\10m\normal\BMW e36\acceleration 0m 10m normal-23-04-2023-19-30-17.json',
        r'..\Datasets\Acceleration\Till reached distance\10m\slow\BMW e36\acceleration 0m 10m slow-23-04-2023-19-19-20.json',

    ]

    #comparação dos resultados
    models = []
    scalers = []
    comparison_dfs = []

    for dataset in datasets:
        print("Load e treino do modelo para:", dataset)
        X_train, X_test, y_train, y_test, scaler = load_dataset(dataset)
        model = build_and_train_model(X_train, X_test, y_train, y_test)
        models.append(model)
        scalers.append(scaler)

        # Carregamento dos dados específicos para classificação de velocidade
        with open(dataset, 'r') as file:
            data = json.load(file)
        classification_result = classify_speed(data)  # Classificar velocidade para os dados atuais
        print(classification_result)

        for i, model in enumerate(models):
            print("\n\nResultados para o Dataset:", datasets[i])
            X_train, X_test, y_train, y_test, scaler = load_dataset(datasets[i])
            comparison_df = predict_and_compare(model, X_test, y_test, scalers[i])
            comparison_dfs.append(comparison_df)
            print(comparison_df)  
            print("\n-----------------------------\n")  

if __name__ == "__main__":
    main()
